In [10]:
%matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.special
from numpy.random import uniform
from sklearn.model_selection import train_test_split


Using matplotlib backend: Qt5Agg


In [2]:
class NeuralNetwork:
    def __init__(self, rate, inputs, hiddens, outputs):
        #добавляем 1 вход под bias
        self.i_count=inputs+1
        self.h_count=hiddens 
        self.o_count=outputs
        #заполняем массивы весов рандомом
        self.w_ih = np.random.normal(0.0, pow(self.h_count, -0.5), (self.h_count, self.i_count))
        self.w_ho = np.random.normal(0.0, pow(self.o_count, -0.5), (self.o_count, self.h_count))
        #learning rate и сигмоид 
        self.lr = rate
        self.activation_function = lambda x: scipy.special.expit(x)
        
    def train(self, inputs_list, targets_list):
        #добавляем 1 вход под bias
        inputs_list=np.concatenate((inputs_list,[1]), axis=0)
        #вектор-столбцы входных данных и правильных ответов
        inputs = np.array(inputs_list, ndmin=2).T 
        targets = np.array(targets_list, ndmin=2).T
        #прямое распространение. сигмоид и линеар.
        hid_results = self.activation_function(np.dot(self.w_ih, inputs))
        out_results = np.dot(self.w_ho, hid_results)
        
        #ошибки вывода
        out_errors = (targets - out_results)
        #ошибки скрытого слоя
        hid_errors = np.dot(self.w_ho.T, out_errors)
        #поправки для весов скрытый-выход
        self.w_ho += self.lr * np.dot(out_errors, np.transpose(hid_results))
        #поправки для весов вход-скрытый
        self.w_ih += self.lr * np.dot((hid_errors * hid_results * (1.0 - hid_results)), np.transpose(inputs))
        
    def query(self, inputs_list):
        #добавляем 1 вход под bias
        inputs_list=np.concatenate((inputs_list,[1]), axis=0)
        #вектор-столбец входных данных
        inputs = np.array(inputs_list, ndmin=2).T 
        #прямое распространение. сигмоид и линеар.
        hid_results = self.activation_function(np.dot(self.w_ih, inputs))
        out_results = np.dot(self.w_ho, hid_results)
        return out_results
    
    def set_lr(self, rate):
        self.lr = rate

In [36]:
net = NeuralNetwork(rate=0.01, inputs=1, hiddens=50, outputs=1)

In [37]:
a = 0.5
b = 1.3

def ApproxFunc(x):
    return a * (b ** x)

In [38]:
# Формирование тренировочной и тестовой выборки

pcenter = 0
prange = 10

train_count = 3000
test_count = 500

period_train = np.arange((pcenter - (prange / 2)), (pcenter + (prange / 2)), (prange / train_count))
period_train_trg = ApproxFunc(period_train)
period_test = np.arange((pcenter - (prange / 2)), (pcenter + (prange / 2)), (prange / test_count))
period_test_trg = ApproxFunc(period_test)


In [39]:
epochs = 100
plt.ion()

for e in range(epochs):
    np.random.shuffle(period_train)
    period_train_trg = ApproxFunc(period_train)
    
    for x in range(train_count):
        net.train([period_train[x]], [period_train_trg[x]])
        
    plt.clf()
    plt.plot(period_test, period_test_trg, 'blue')
    MSE = 0
    test_results = np.zeros(test_count)
    
    for x in range(test_count):
        result = net.query([period_test[x]])
        MSE += (ApproxFunc(period_test[x]) - result) ** 2
        test_results[x] = result
    
    plt.plot(period_test, test_results, 'red')
    plt.pause(1)
    print(MSE / test_count)

[[0.01489688]]
[[0.00237839]]
[[0.00367578]]
[[0.00144336]]
[[0.00117291]]
[[0.00109385]]
[[0.00125136]]
[[0.00187532]]
[[0.00098169]]
[[0.00078472]]
[[0.0007265]]
[[0.00075237]]
[[0.00089477]]
[[0.00070557]]
[[0.00075225]]
[[0.00072681]]
[[0.00065516]]
[[0.00053671]]


KeyboardInterrupt: 